# old_positions_data.xlsx to csv cleanup:
This process is performed in order to prepare the data for the transformation across the timeline, so the warehouse will represent as best possible to how it looked like on the 1.1.2021.

The following steps are performed:
1. Format dates to match the activity_data table.
2. Remove rows which contain items in problematic locations (`'sort', 'cd', 'kir', 'flawed', 'rl'`).
3. Merge the old_positions_data table with the item sized and cell sizes.
4. Fill in the volumes for items which are missing.

In [ ]:
import pandas as pd
raw_data_route = '../data/raw/'
old_positions_data = pd.read_excel(f'{raw_data_route}old_positions_data.xlsx', sheet_name="position")
item_sizes = pd.read_excel(f'{raw_data_route}item_sizes.xlsx', sheet_name="Sheet1")
# read the 'Sheet1' from the excel file 'cell_sizes.xlsx' into a DataFrame, and make sure that the 'location' and 'aisle',  are objects:
cell_sizes = pd.read_excel(f'{raw_data_route}cell_sizes.xlsx', sheet_name="Sheet1", dtype={'location': object, 'aisle': object})

In [ ]:
# Fix the old_positions_datas worksheet:
column_names = old_positions_data.columns
date_columns = [name for name in column_names if 'date' in name]
date_columns
old_positions_data[date_columns] = pd.to_datetime(old_positions_data[date_columns].stack()).unstack()  # Stack and unstack are needed because to_datetime works with Series but not with DataFrame.
old_positions_data[['quantity counted']] = old_positions_data[['quantity counted']].astype(float)
# turn all the old_positions_data columns which contain strings to uppercase:
old_positions_data = old_positions_data.apply(lambda x: x.str.upper() if x.dtype == "object" else x)
old_positions_data

In [ ]:
# Remove all items which aren't placed on shelves in an indexed pattern:
problematic_location_names = ['sort', 'cd', 'kir', 'flawed', 'rl']
for name in problematic_location_names: # Remove all problematic places from the DF so we can work with items we can manage
  old_positions_data = old_positions_data[~old_positions_data.location.str.contains(name, case=False)]
old_positions_data

In [ ]:
cell_sizes.head()

In [ ]:
old_positions_data_with_item_sizes = pd.merge(old_positions_data, item_sizes, how='left', on='id')
old_positions_data_with_item_sizes

In [ ]:
# merge the united_old_positions_data with the cell_sizes worksheet, using the 'location' column as the key:
united_old_positions_data = pd.merge(old_positions_data_with_item_sizes.drop(labels=['aisle'], axis=1), cell_sizes, how='left', on='location')
united_old_positions_data.head()

In [ ]:
# Check for the number of empty values in the 'volume' column:
united_old_positions_data['volume'].isna().sum()

In [ ]:
# from the united_old_positions_data, select only the rows which have a NaN in the 'volume' column:
# then calculate the missing values using the following logic:
# if there is more than one row with the same 'location' and 'aisle' values, then make the volume equal to the corresponding 'cubic' value in the cell_sizes worksheet divided by the number of rows with the same 'location' and 'aisle' values:
# if the 'quantity counted' cell is 1.0, then make the volume equal to the corresponding 'cubic' value in the cell_sizes worksheet:
# if the 'quantity counted' cell is greater than 1.0, then make the volume equal to the corresponding 'cubic' value in the cell_sizes worksheet divided by the 'quantity counted' value:
def calculate_volume(row):
  if pd.isna(row['volume']):
    if row['quantity counted'] == 1.0:
      return row['cubic']
    else:
      return row['cubic'] / row['quantity counted']
  else:
    return row['volume']

In [ ]:
# use the calculate_volume function on the united_old_positions_data DataFrame:
united_old_positions_data['volume'] = united_old_positions_data.apply(calculate_volume, axis=1)
# Check for the number of empty values in the 'volume' column, post function application:
united_old_positions_data['volume'].isna().sum()

In [ ]:
united_old_positions_data

In [ ]:
united_old_positions_data[united_old_positions_data['location'] == '40260901'].sort_values(by=['symbol'])

In [ ]:
# Take the united_old_positions_data DataFrame and group it by the 'location' column, then count how many rows are in each group:
grouped_old_positions_data = united_old_positions_data[['id', 'location']].groupby('location').count().sort_values(by=['id'], ascending=False)
# show only the rows from grouped_old_positions_data with a count greater than 1:
grouped_old_positions_data[grouped_old_positions_data['id'] > 50]
# create a plot showing the distribution of the grouped_old_positions_data DataFrame:

In [ ]:
# count how many unique 'location' values are in the united_old_positions_data DataFrame:
united_old_positions_data['location'].nunique()

In [ ]:
old_positions_data['location'].nunique()

In [ ]:
cell_sizes.nunique()

In [ ]:
# Check how many values are overlapping between the old_positions_data 'location' and cell_sizes 'location' columns:
len(set(old_positions_data['location'].unique()).intersection(set(cell_sizes['location'].unique())))

In [ ]:
13483-5139

In [ ]:
# Get all rows from the old_positions_data DataFrame where the 'aisle' column is equal to 10 or 11:
old_positions_data[(old_positions_data['aisle'] == 10) | (old_positions_data['aisle'] == 11)]

In [ ]:
# in united_old_positions_data DataFrame, count for each 'id' how many different 'location' values there are:
united_old_positions_data[['id', 'location']].groupby('id').nunique().sort_values(by=['location'], ascending=False)

In [ ]:
# from united_old_positions_data get all rows which have an 'id' of 'AAAAA-28953':
united_old_positions_data[united_old_positions_data['id'] == 'AAAAA-28953']